In [1]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [2]:
rerun = False

In [3]:
### Tools' setting ###
# PATHS
owl_version = '1.1.0'
owl       = 'owl-'+owl_version+'/bin/ltl2ldba %[MWi]f'
nba2ldba  = 'ltl2tgba --deterministic -f %f --ba | owl-'+owl_version+'/bin/nba2ldba'
seminator = 'ltl2tgba --deterministic -f %f | ./seminator'
cy        = 'ltl2tgba --deterministic -f %f --ba | ./seminator --cy'
### Stripping command for ltl2ldba ###
# Ltl2ldba prints carbage collection messages to stdout
# which causes errors in ltlcross.
# We throw everything until the line HOA v1
cut  = ' --cd'
simp_sd = ' | autfilt --small --tgba'
simp_cd = ' | autfilt -x simul=1,ba-simul=1 --small --tgba'
nos   = ' -s0' # disables Spot's simplifications used in Seminator
end  = ' > %O'

### Ltlcross runner configuration ###
tools = {## Cut-deterministic
         'cd.ltl2ldba.no' : owl + end,
         'cd.ltl2ldba.yes'    : owl + simp_cd + end,
         'cd.seminator.no': seminator + nos + cut + end,
         'cd.seminator.yes'   : seminator + cut + end,
         'cd.cy.no'       : cy + nos + cut + end,     
         'cd.cy.yes'          : cy + cut + end,
         ### Semi-deterministic
         'sd.ltl2ldba.no' : owl + ' -n' + end,
         'sd.ltl2ldba.yes'    : owl + ' -n' + simp_sd + end,
         'sd.nba2ldba.no' : nba2ldba + end,
         'sd.nba2ldba.yes'    : nba2ldba + simp_sd + end,
         'sd.seminator.no': seminator + nos + end,
         'sd.seminator.yes'   : seminator + end,
         'sd.cy.no'       : cy + nos + end,  
         'sd.cy.yes'          : cy + end,
        }
sd_tools = ('sd.cy.no','sd.cy.yes',
            'sd.seminator.no','sd.seminator.yes',
            'sd.ltl2ldba.no','sd.ltl2ldba.yes',
            'sd.nba2ldba.no','sd.nba2ldba.yes')
cd_tools = ('cd.cy.no','cd.cy.yes',
            'cd.seminator.no','cd.seminator.yes',
            'cd.ltl2ldba.no','cd.ltl2ldba.yes')
### Formula files ###
det_files = ['formulae/literature_det.ltl']
nondet_files = ['formulae/literature_nondet.ltl']
det_r_files = ['formulae/rand_det.ltl']
nondet_r_files = ['formulae/rand_nondet.ltl']
### File with measured statistics ###
det_res_file = "sdba_literature_det.csv"
nondet_res_file = "sdba_literature_nondet.csv"
det_r_res_file = "sdba_random_det.csv"
nondet_r_res_file = "sdba_random_nondet.csv"
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = sd_tools + cd_tools

In [4]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [5]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [6]:
data = pd.DataFrame()
for (name,r) in runners.items():
    data = data.append(pd.DataFrame(r.cummulative(),columns=[name]).transpose())
data

tool,cd.cy.no,cd.cy.yes,cd.ltl2ldba.no,cd.ltl2ldba.yes,cd.seminator.no,cd.seminator.yes,sd.cy.no,sd.cy.yes,sd.ltl2ldba.no,sd.ltl2ldba.yes,sd.nba2ldba.no,sd.nba2ldba.yes,sd.seminator.no,sd.seminator.yes
literature_nd,756.0,423.0,421.0,361.0,542.0,348.0,428.0,336.0,617.0,327.0,738.0,475.0,373.0,302.0
random_sd,750.0,728.0,1495.0,1275.0,734.0,712.0,720.0,720.0,1228.0,784.0,850.0,774.0,704.0,704.0
random_nd,2350.0,1360.0,1387.0,1038.0,2025.0,1140.0,1417.0,1083.0,1314.0,804.0,3657.0,1875.0,1231.0,937.0
literature_det,600.0,600.0,1039.0,809.0,556.0,556.0,600.0,600.0,1277.0,855.0,600.0,600.0,556.0,556.0
literature_sd,13.0,13.0,53.0,40.0,13.0,13.0,13.0,13.0,41.0,17.0,17.0,13.0,13.0,13.0
random_det,426.0,426.0,570.0,497.0,413.0,413.0,426.0,426.0,638.0,446.0,426.0,426.0,413.0,413.0
random_cd,505.0,505.0,732.0,649.0,463.0,463.0,505.0,505.0,733.0,539.0,863.0,634.0,463.0,463.0
literature_cd,207.0,207.0,612.0,488.0,194.0,194.0,207.0,207.0,838.0,341.0,377.0,240.0,194.0,194.0


Get the number of formulas such that all tools finished within timeout for each category.

In [7]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

,cd..n,sd..n
literature_nd,23,22
random_sd,100,100
random_nd,100,100
literature_det,149,149
literature_sd,3,3
random_det,100,100
random_cd,100,100
literature_cd,46,46


Converts the DataFrame to use MultiIndices for rows and columns

In [8]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [9]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [10]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

cd                                                      sd  \
                          cy         ltl2ldba         seminator                
                   n      no     yes       no     yes        no     yes    n   
origin     type                                                                
random     det   100   426.0   426.0    570.0   497.0     413.0   413.0  100   
           cd    100   505.0   505.0    732.0   649.0     463.0   463.0  100   
           sd    100   750.0   728.0   1495.0  1275.0     734.0   712.0  100   
           nd    100  2350.0  1360.0   1387.0  1038.0    2025.0  1140.0  100   
literature det   149   600.0   600.0   1039.0   809.0     556.0   556.0  149   
           cd     46   207.0   207.0    612.0   488.0     194.0   194.0   46   
           sd      3    13.0    13.0     53.0    40.0      13.0    13.0    3   
           nd     23   756.0   423.0    421.0   361.0     542.0   348.0   22   

                                                                            \
                     cy         ltl2ldba        nba2ldba         seminator   
                     no     yes       no    yes       no     yes        no   
origin     type                                                              
random     det    426.0   426.0    638.0  446.0    426.0   426.0     413.0   
           cd     505.0   505.0    733.0  539.0    863.0   634.0     463.0   
           sd     720.0   720.0   1228.0  784.0    850.0   774.0     704.0   
           nd    1417.0  1083.0   1314.0  804.0   3657.0  1875.0    1231.0   
literature det    600.0   600.0   1277.0  855.0    600.0   600.0     556.0   
           cd     207.0   207.0    838.0  341.0    377.0   240.0     194.0   
           sd      13.0    13.0     41.0   17.0     17.0    13.0      13.0   
           nd     428.0   336.0    617.0  327.0    738.0   475.0     373.0   

                        
                        
                   yes  
origin     type         
random     det   413.0  
           cd    463.0  
           sd    704.0  
           nd    937.0  
literature det   556.0  
           cd    194.0  
           sd     13.0  
           nd    302.0

In [11]:
cd = tmp.cd
sd = tmp.sd

In [12]:
cd

cy         ltl2ldba         seminator        
                   n      no     yes       no     yes        no     yes
origin     type                                                        
random     det   100   426.0   426.0    570.0   497.0     413.0   413.0
           cd    100   505.0   505.0    732.0   649.0     463.0   463.0
           sd    100   750.0   728.0   1495.0  1275.0     734.0   712.0
           nd    100  2350.0  1360.0   1387.0  1038.0    2025.0  1140.0
literature det   149   600.0   600.0   1039.0   809.0     556.0   556.0
           cd     46   207.0   207.0    612.0   488.0     194.0   194.0
           sd      3    13.0    13.0     53.0    40.0      13.0    13.0
           nd     23   756.0   423.0    421.0   361.0     542.0   348.0

In [13]:
sd

cy         ltl2ldba        nba2ldba          \
                   n      no     yes       no    yes       no     yes   
origin     type                                                         
random     det   100   426.0   426.0    638.0  446.0    426.0   426.0   
           cd    100   505.0   505.0    733.0  539.0    863.0   634.0   
           sd    100   720.0   720.0   1228.0  784.0    850.0   774.0   
           nd    100  1417.0  1083.0   1314.0  804.0   3657.0  1875.0   
literature det   149   600.0   600.0   1277.0  855.0    600.0   600.0   
           cd     46   207.0   207.0    838.0  341.0    377.0   240.0   
           sd      3    13.0    13.0     41.0   17.0     17.0    13.0   
           nd     22   428.0   336.0    617.0  327.0    738.0   475.0   

                seminator         
                       no    yes  
origin     type                   
random     det      413.0  413.0  
           cd       463.0  463.0  
           sd       704.0  704.0  
           nd      1231.0  937.0  
literature det      556.0  556.0  
           cd       194.0  194.0  
           sd        13.0   13.0  
           nd       373.0  302.0

## Timeouts

In [14]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

cd                                     sd                \
                  cy      ltl2ldba      seminator        cy      ltl2ldba   
                  no  yes       no  yes        no  yes   no  yes       no   
origin     type                                                             
literature nd    0.0  0.0      0.0  0.0       0.0  0.0  0.0  0.0      0.0   

                                                   
                     nba2ldba      seminator       
                 yes       no  yes        no  yes  
origin     type                                    
literature nd    0.0      0.0  0.0       0.0  1.0

For exactly one formula we have encountered a timeout. Get numbers for the timeouted formula and get the data into the same shape as for cummulative numbers.

In [15]:
# Get the numbers
r = runners['literature_nd'].values
fail = r.states[r.exit_status['sd.seminator.yes'] != 'ok'].loc[:,sd_tools]
fail.columns = fail.columns.str.replace('sd.','')
# Shape the data
tmp = fail.reset_index()
tmp.drop(['formula','form_id'],axis=1,inplace=True)
tmp['source'] = ['lit. (T/O)_nd']
tmp = tmp.set_index('source')
tmp.index.name = None
tmp['.n'] = len(fail)
sd_fail = make_hierarchical(tmp)

Finally, we append the data into the table for semi-determinization

In [16]:
sd.append(sd_fail).fillna('---')

cy         ltl2ldba        nba2ldba          \
                   n      no     yes       no    yes       no     yes   
origin     type                                                         
random     det   100   426.0   426.0    638.0  446.0    426.0   426.0   
           cd    100   505.0   505.0    733.0  539.0    863.0   634.0   
           sd    100   720.0   720.0   1228.0  784.0    850.0   774.0   
           nd    100  1417.0  1083.0   1314.0  804.0   3657.0  1875.0   
literature det   149   600.0   600.0   1277.0  855.0    600.0   600.0   
           cd     46   207.0   207.0    838.0  341.0    377.0   240.0   
           sd      3    13.0    13.0     41.0   17.0     17.0    13.0   
           nd     22   428.0   336.0    617.0  327.0    738.0   475.0   
lit. (T/O) nd      1    99.0    67.0     49.0   49.0    131.0    98.0   

                seminator       
                       no  yes  
origin     type                 
random     det      413.0  413  
           cd       463.0  463  
           sd       704.0  704  
           nd      1231.0  937  
literature det      556.0  556  
           cd       194.0  194  
           sd        13.0   13  
           nd       373.0  302  
lit. (T/O) nd        99.0  ---

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [17]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.


In [18]:
display(cd)

cy         ltl2ldba         seminator        
                   n      no     yes       no     yes        no     yes
origin     type                                                        
random     det   100   426.0   426.0    570.0   497.0     413.0   413.0
           cd    100   505.0   505.0    732.0   649.0     463.0   463.0
           sd    100   750.0   728.0   1495.0  1275.0     734.0   712.0
           nd    100  2350.0  1360.0   1387.0  1038.0    2025.0  1140.0
literature det   149   600.0   600.0   1039.0   809.0     556.0   556.0
           cd     46   207.0   207.0    612.0   488.0     194.0   194.0
           sd      3    13.0    13.0     53.0    40.0      13.0    13.0
           nd     23   756.0   423.0    421.0   361.0     542.0   348.0

In [19]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
Reductions of the automaton produced by Seminator for one formula from literature did not finish on time (1m). We list the results for the other tools for this particular formula in the last row.
''',raw=True)

### Comparison of tools producing semi-deterministic automata
Reductions of the automaton produced by Seminator for one formula from literature did not finish on time (1m). We list the results for the other tools for this particular formula in the last row.


In [20]:
display(sd.append(sd_fail))

cy         ltl2ldba        nba2ldba          \
                   n      no     yes       no    yes       no     yes   
origin     type                                                         
random     det   100   426.0   426.0    638.0  446.0    426.0   426.0   
           cd    100   505.0   505.0    733.0  539.0    863.0   634.0   
           sd    100   720.0   720.0   1228.0  784.0    850.0   774.0   
           nd    100  1417.0  1083.0   1314.0  804.0   3657.0  1875.0   
literature det   149   600.0   600.0   1277.0  855.0    600.0   600.0   
           cd     46   207.0   207.0    838.0  341.0    377.0   240.0   
           sd      3    13.0    13.0     41.0   17.0     17.0    13.0   
           nd     22   428.0   336.0    617.0  327.0    738.0   475.0   
lit. (T/O) nd      1    99.0    67.0     49.0   49.0    131.0    98.0   

                seminator         
                       no    yes  
origin     type                   
random     det      413.0  413.0  
           cd       463.0  463.0  
           sd       704.0  704.0  
           nd      1231.0  937.0  
literature det      556.0  556.0  
           cd       194.0  194.0  
           sd        13.0   13.0  
           nd       373.0  302.0  
lit. (T/O) nd        99.0    NaN